In [1]:
import os

os.environ["SEED"] = "0"
os.environ["TRAIN_DIR"] = "TRAIN"

import numpy as np
from tqdm.auto import tqdm

from src.aac.AAC import AAC
from src.aac.SurrogateEstimator import Estimator1
from src.constant import DATA_DIR, DATABASE_DIR
from src.database import DB
from src.database.queries import get_model_training_data
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.log import logger

In [2]:
train_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TRAIN" / "index.json",
    cut_off_cost=100,
    cut_off_time=10,
    n=5,
)
test_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TEST" / "index.json",
    cut_off_cost=1000,
    cut_off_time=100,
    n=250,
)

In [3]:
for instance in train_instances:
    instance.cut_off_time = round(10 * ((instance.n_cities / 600) ** 2.2), 2)
    instance.cut_off_cost = 10 * instance.cut_off_time

In [4]:
SOLVERS_N = 2
ATTEMPTS_N = 3
MAX_ITER = 2

solvers = []
largest_marginal_contribution_solver = None

for solver_i in range(SOLVERS_N):
    logger.info(f"Solver {solver_i + 1}/{SOLVERS_N}")

    best_cost = np.inf
    best_solver = None
    attempt_solvers = []

    for attempt_i in range(ATTEMPTS_N):
        logger.info(f"Attempt {attempt_i + 1}/{ATTEMPTS_N}")

        if largest_marginal_contribution_solver is not None:
            new_solver = largest_marginal_contribution_solver.copy()
        else:
            new_solver = TSP_LKH_Solver()

        iteration_solvers = solvers + [new_solver]

        portfolio = Portfolio.from_iterable(iteration_solvers)
        aac = AAC(
            portfolio=portfolio,
            instance_list=train_instances,
            prefix=f"config;solver={solver_i+1};attempt={attempt_i+1}",
            max_iter=MAX_ITER,
            i=solver_i,
            calculate_features=False,
        )
        portfolio = aac.configure()
        result = portfolio.evaluate(  # fix cut-off times before validation
            instance_list=train_instances,
            prefix=f"validate;solver={solver_i+1};attempt={attempt_i+1}",
            cache=True,
        )
        attempt_solvers.append(portfolio[solver_i])
        logger.info(
            f"Attempt {attempt_i + 1}/{ATTEMPTS_N}: cost = {result.cost:.2f}"
        )
        if result.cost < best_cost:
            best_cost = result.cost
            best_solver = portfolio[solver_i]

    solvers.append(best_solver)
    logger.info(f"Solver {solver_i + 1}/{SOLVERS_N}: best cost = {best_cost:.2f}")

    if solver_i < SOLVERS_N - 1:
        largest_marginal_contribution_solver = None
        best_cost = np.inf
        for attempt_i, solver in enumerate(attempt_solvers):
            if solver != best_solver:
                portfolio = Portfolio.from_iterable(solvers + [solver])
                result = portfolio.evaluate(
                    instance_list=train_instances,
                    prefix=f"largest_marginal_contribution;attempt={attempt_i+1}",
                    cache=True,
                )
                if result.cost < best_cost:
                    best_cost = result.cost
                    largest_marginal_contribution_solver = solver

[2025-04-28 23:22:29] INFO      Solver 1/2
[2025-04-28 23:22:29] INFO      Attempt 1/3
[2025-04-28 23:22:29] DEBUG     AAC(prefix=config;solver=1;attempt=1, iter=1/2)
[2025-04-28 23:22:29] DEBUG     Portfolio.evaluate(config;solver=1;attempt=1;aac_iter=1)
[2025-04-28 23:22:29] DEBUG     Portfolio(size=1)[Solver(id=52980316141715150)]
[2025-04-28 23:22:29] DEBUG     solve(prefix=config;solver=1;attempt=1;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/TRAIN/cluster_netgen/000.tsp))
[2025-04-28 23:22:29] DEBUG     solve(prefix=config;solver=1;attempt=1;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/TRAIN/compression/000.tsp))
[2025-04-28 23:22:29] DEBUG     solve(prefix=config;solver=1;attempt=1;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/TRAIN/expansion/000.tsp))
[2025-04-28 23:22:29] DEBUG     solve(prefix=config;solver=1;attempt=1;aac_iter=1, solver=Solver(id=52980316141715

In [5]:
DB().get_results() #.loc[lambda x: (x["cached"] == 0) & (x["cut_off_time"] == 2)]

,id,prefix,solver_id,instance_id,cost,time,cut_off_cost,cut_off_time,cached,surrogate,error
0,config;solver=1;attempt=1;aac_iter=1_529803161...,config;solver=1;attempt=1;aac_iter=1,52980316141715150,1877163610474807747,1.29,1.29,30.4,3.04,0,0,0
1,config;solver=1;attempt=1;aac_iter=1_529803161...,config;solver=1;attempt=1;aac_iter=1,52980316141715150,1099564706851039575,4.40,0.44,4.4,0.44,0,0,0
2,config;solver=1;attempt=1;aac_iter=1_529803161...,config;solver=1;attempt=1;aac_iter=1,52980316141715150,1578235657227061094,6.90,0.69,6.9,0.69,0,0,0
3,config;solver=1;attempt=1;aac_iter=1_529803161...,config;solver=1;attempt=1;aac_iter=1,52980316141715150,1630180243408159957,52.30,5.23,52.3,5.23,0,0,0
4,config;solver=1;attempt=1;aac_iter=1_529803161...,config;solver=1;attempt=1;aac_iter=1,52980316141715150,1170191481226940335,59.30,5.93,59.3,5.93,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
150,validate;solver=2;attempt=3_176228367859327934...,validate;solver=2;attempt=3,176228367859327934,1099564706851039575,0.35,0.00,4.4,0.44,1,0,0
151,validate;solver=2;attempt=3_205857259882012742...,validate;solver=2;attempt=3,2058572598820127428,1170191481226940335,0.33,0.00,59.3,5.93,1,0,0
152,validate;solver=2;attempt=3_176228367859327934...,validate;solver=2;attempt=3,176228367859327934,1170191481226940335,59.30,0.00,59.3,5.93,1,0,0
153,validate;solver=2;attempt=3_205857259882012742...,validate;solver=2;attempt=3,2058572598820127428,1877163610474807747,1.51,0.00,30.4,3.04,1,0,0
